<a href="https://colab.research.google.com/github/kostas-panagiotakis/Bayesian-Optimization/blob/main/Copy_of_BarbenHeimer_Review_RAG_with_AIMS_and_Deci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BarbenHeimer with LLaMA 2


![image](https://images.lumacdn.com/cdn-cgi/image/format=auto,fit=cover,dpr=1,quality=75,width=960,height=480/event-covers/87/1f6d4850-0231-4bd9-92e8-af6b45c18d7a)


In the following notebook we'll be discussing Retrieval Augmented Generation - and how to leverage Meta's neweset LLM, LLaMA 2 as the engine!

### Pre-task Work

All we really need to do to get started is to get our prerequisites!

We'll be leveraging `langchain` and `llama 2` today.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [LLaMA 2](https://huggingface.co/blog/llama2)

In [1]:
!pip install -U -q "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1"
!pip install jq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Task 1: Data Preparation

In this task we'll be collecting, and then parsing, our data.

In [2]:
import pandas as pd
from datasets import load_dataset
from langchain_community.document_loaders import JSONLoader
import json
from pathlib import Path
from pprint import pprint


In [3]:
 # Load the dataset
dataset = load_dataset("enesxgrahovac/the-feynman-lectures-on-physics")
dataset
train = dataset['train']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/641 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/enesxgrahovac___parquet/enesxgrahovac--the-feynman-lectures-on-physics-ac06d176413574ad/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

#### Data Parsing

Now that we have our data - let's go ahead and start parsing it into a more usable format for LangChain!

We'll be using the `CSVLoader` for this application.

Check out the docs here:
- [CSVLoader](https://python.langchain.com/docs/integrations/document_loaders/csv)

In [4]:
# Convert the dataset into df format
df = pd.DataFrame(train)
df = df.sort_values(by=['book_volume','chapter_number','section_number'])
df

,book_volume,book_title,chapter_number,chapter_title,section_number,section_title,section_text
0,1,,1,Atoms in Motion,1,Introduction,This two-year course in physics is presented f...
39,1,,10,Conservation of Momentum,1,Newton’s Third Law,"On the basis of Newton’s second law of motion,..."
40,1,,10,Conservation of Momentum,2,Conservation of momentum,Now what are the interesting consequences of t...
41,1,,10,Conservation of Momentum,3,Momentum conserved!,We can verify the above assumptions experiment...
42,1,,10,Conservation of Momentum,4,Momentum and energy,All the foregoing examples are simple cases wh...
...,...,...,...,...,...,...,...
555,3,,9,The Ammonia Maser,2,The molecule in a static electric field,If the ammonia molecule is in either of the tw...
556,3,,9,The Ammonia Maser,3,Transitions in a time-dependent field,"In the ammonia maser, the beam with molecules ..."
557,3,,9,The Ammonia Maser,4,Transitions at resonance,Let’s take the case of perfect resonance first...
558,3,,9,The Ammonia Maser,5,Transitions off resonance,"Finally, we would like to find out how the sta..."


In [5]:
# Convert df to json
feynman_json = df.to_json('output.json', orient='records')
file_path='/content/output.json'
feynman_data = json.loads(Path(file_path).read_text())
print(feynman_data[0])

{'book_volume': '1', 'book_title': '', 'chapter_number': '1', 'chapter_title': 'Atoms in Motion', 'section_number': '1', 'section_title': 'Introduction', 'section_text': 'This two-year course in physics is presented from the point of view that you, the reader, are going to be a physicist. This is not necessarily the case of course, but that is what every professor in every subject assumes! If you are going to be a physicist, you will have a lot to study: two hundred years of the most rapidly developing field of knowledge that there is. So much knowledge, in fact, that you might think that you cannot learn all of it in four years, and truly you cannot; you will have to go to graduate school too! Surprisingly enough, in spite of the tremendous amount of work that has been done for all this time it is possible to condense the enormous mass of results to a large extent—that is, to find laws which summarize all our knowledge. Even so, the laws are so hard to grasp that it is unfair to you t

In [6]:
pprint(feynman_data[0:2])

[{'book_title': '',
  'book_volume': '1',
  'chapter_number': '1',
  'chapter_title': 'Atoms in Motion',
  'section_number': '1',
  'section_text': 'This two-year course in physics is presented from the point '
                  'of view that you, the reader, are going to be a physicist. '
                  'This is not necessarily the case of course, but that is '
                  'what every professor in every subject assumes! If you are '
                  'going to be a physicist, you will have a lot to study: two '
                  'hundred years of the most rapidly developing field of '
                  'knowledge that there is. So much knowledge, in fact, that '
                  'you might think that you cannot learn all of it in four '
                  'years, and truly you cannot; you will have to go to '
                  'graduate school too! Surprisingly enough, in spite of the '
                  'tremendous amount of work that has been done for all this '
           

In [7]:
loader = JSONLoader(
    file_path='/content/output.json',
    jq_schema='.[].section_text',
    text_content=False)

feynman_data_loaded = loader.load()

In [8]:
len(feynman_data_loaded)

641

Now that we have collected our review information into a loader - we can go ahead and chunk the reviews into more manageable pieces.

We'll be leveraging the `RecursiveCharacterTextSplitter` for this task today.

While splitting our text seems like a simple enough task - getting this correct/incorrect can have massive downstream impacts on your application's performance.

You can read the docs here:
- [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)

> ### HINT:
>It's always worth it to check out the LangChain source code if you're ever in a bind - for instance, if you want to know how to transform a set of documents, check it out [here](https://github.com/langchain-ai/langchain/blob/5e9687a196410e9f41ebcd11eb3f2ca13925545b/libs/langchain/langchain/text_splitter.py#L268C18-L268C18)

# Parameter optimization


In [35]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # the character length of the chunk
    chunk_overlap = 100, # the character length of the overlap between chunks
    length_function = len, # the length function - in this case, character length (aka the python len() fn.)
)

In [36]:
feynman_documents = text_splitter.transform_documents(feynman_data_loaded)

In [37]:
len(feynman_documents)

5406

In [38]:
feynman_documents[0]

Document(page_content='This two-year course in physics is presented from the point of view that you, the reader, are going to be a physicist. This is not necessarily the case of course, but that is what every professor in every subject assumes! If you are going to be a physicist, you will have a lot to study: two hundred years of the most rapidly developing field of knowledge that there is. So much knowledge, in fact, that you might think that you cannot learn all of it in four years, and truly you cannot; you will have to go to graduate school too! Surprisingly enough, in spite of the tremendous amount of work that has been done for all this time it is possible to condense the enormous mass of results to a large extent—that is, to find laws which summarize all our knowledge. Even so, the laws are so hard to grasp that it is unfair to you to start exploring this tremendous subject without some kind of map or outline of the relationship of one part of the subject of science to another. 

With our documents transformed into more manageable sizes, and with the correct metadata set-up, we're now ready to move on to creating our VectorStore!

### Task 2: Creating an "Index"

The term "index" is used largely to mean: Structured documents parsed into a useful format for querying, retrieving, and use in the LLM application stack.

#### Selecting Our VectorStore

There are a number of different VectorStores, and a number of different strengths and weaknesses to each.

In this notebook, we will be keeping it very simple by leveraging [Facebook AI Similarity Search](https://ai.meta.com/tools/faiss/#:~:text=FAISS%20(Facebook%20AI%20Similarity%20Search,more%20scalable%20similarity%20search%20functions.), or `FAISS`.

# Analyze different vector stores and how they impact performances

In [24]:
# !pip install -q -U faiss-cpu tiktoken sentence-transformers

We're going to be setting up our VectorStore with the OpenAI embeddings model. While this embeddings model does not need to be consistent with the LLM selection, it does need to be consistent between embedding our index and embedding our queries over that index.

While we don't have to worry too much about that in this example - it's something to keep in mind for more complex applications.

We're going to leverage a [`CacheBackedEmbeddings`](https://python.langchain.com/docs/modules/data_connection/caching_embeddings )flow to prevent us from re-embedding similar queries over and over again.

Not only will this save time, it will also save us precious embedding tokens, which will reduce the overall cost for our application.

>#### Note:
>The overall cost savings needs to be compared against the additional cost of storing the cached embeddings for a true cost/benefit analysis. If your users are submitting the same queries often, though, this pattern can be a massive reduction in cost.

In [39]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.from_documents(feynman_documents, embedder)

In [52]:
core_embeddings_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

Now that we've created the VectorStore, we can check that it's working by embedding a query and retrieving passages from our reviews that are close to it.

In [54]:
query = "Give me the definition of Energy in terms of work"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 1)

for page in docs:
  print(page.page_content)

also called a watt (W). If we multiply watts by time, the result is the work done. The work done by the electrical company in our houses, technically, is equal to the watts times the time. That is where we get things like kilowatt hours, $1000$ watts times $3600$ seconds, or $3.6\times10^6$ joules.  Now we take another example of the law of conservation of energy. Consider an object which initially has kinetic energy and is moving very fast, and which slides against the floor with friction. It stops. At the start the kinetic energy is not zero, but at the end it is zero; there is work done by the forces, because whenever there is friction there is always a component of force in a direction opposite to that of the motion, and so energy is steadily lost. But now let us take a mass on the end of a pivot swinging in a vertical plane in a gravitational field with no friction. What happens here is different, because when the mass is going up the force is downward, and when it is coming


Let's see how much time the `CacheBackedEmbeddings` pattern saves us:

In [ ]:
%%timeit -n 1 -r 1
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

9.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

8.16 ms ± 33.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As we can see, even over a significant number of runs - the cached query is significantly faster than the first instance of the query!

With that, we're ready to move onto Task 3!

### Task 3: Building a Retrieval Chain

In this task, we'll be making a Retrieval Chain which will allow us to ask semantic questions over our data.

This part is rather abstracted away from us in LangChain and so it seems very powerful.

Be sure to check the documentation, the source code, and other provided resources to build a deeper understanding of what's happening "under the hood"!

#### A Basic RetrievalQA Chain

We're going to leverage `return_source_documents=True` to ensure we have proper sources for our reviews - should the end user want to verify the reviews themselves.

Hallucinations [are](https://arxiv.org/abs/2202.03629) [a](https://arxiv.org/abs/2305.15852) [massive](https://arxiv.org/abs/2303.16104) [problem](https://arxiv.org/abs/2305.18248) in LLM applications.

Though it has been tenuously shown that using Retrieval Augmentation [reduces hallucination in conversations](https://arxiv.org/pdf/2104.07567.pdf), one sure fire way to ensure your model is not hallucinating in a non-transparent way is to provide sources with your responses. This way the end-user can verify the output.

#### Our LLM

In this notebook, we're going to leverage Meta's LLaMA 2!

Specifically, we'll be using: `meta-llama/Llama-2-13b-chat-hf`

That's right, a 13B parameter model that we're going to run on *less than* 15GB of GPU RAM.

More information on this model can be found [here](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf)

# Compare two models the 13 vs the 7 bill params

In [55]:
!pip install huggingface-hub -q

In [56]:
from huggingface_hub import notebook_login

notebook_login()

We will be leveraging Tim Dettmer's `bitsandbytes` as well as `accelerate` and `transformers` from Hugging Face to make our model as small as possible. The overall quality of the model is fairly well retained!

In [58]:
import torch
import transformers

model_id = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

model.eval()

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-13b-chat-hf.
403 Client Error. (Request ID: Root=1-66477062-7e00327820faf85d6b908fe3;03bae6f2-8424-4116-a6d1-c60ac9e411e0)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-13b-chat-hf/resolve/main/config.json.
Your request to access model meta-llama/Llama-2-13b-chat-hf is awaiting a review from the repo authors.

In [79]:
import torch
from transformers import AutoModelForCausalLM, AutoConfig
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Specify the model name or ID of a publicly accessible model
model_id = "gpt2"

# Load model configuration
model_config = AutoConfig.from_pretrained(model_id)

# Load the model
model = GPT2LMHeadModel.from_pretrained(model_id)

# Set the model to evaluation mode
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [80]:
tokenizer = GPT2Tokenizer.from_pretrained(model_id)


Now we need to pack it into a `pipeline` for compatability with `langchain`!

In [81]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.3,
    max_new_tokens=256
)

In [82]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

Now we can set up our chain.

In [83]:
retriever = vector_store.as_retriever()

In [84]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

Now that it's set-up, let's test it out!

In [85]:
qa_with_sources_chain({"query" : "Give me the definition of energy"})

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




> Entering new RetrievalQA chain...


IndexError: index out of range in self

In [ ]:
qa_with_sources_chain({"query" : "Do reviewers consider this movie Kenough?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do reviewers consider this movie Kenough?',
 'result': ' Based on the reviews provided, it seems that the reviewers have mixed opinions about the movie. Some reviewers, such as eoinageary and hamsterination, have positive reviews and rate the movie 8 and 6 out of 10, respectively. However, other reviewers, such as brianjohnson-20043 and holarsch, have more negative reviews and rate the movie 4 and 6 out of 10, respectively. Therefore, it cannot be determined whether the reviewers consider the movie Kenough or not.',
 'source_documents': [Document(page_content=': 37\nReview_Date: 23 July 2023\nAuthor: eoinageary\nRating: 8\nReview_Title: I am Kenough\nReview: So I went into the movie with little to no expectations and I was pleasantly impressed with the movie overall.\nReview_Url: /review/rw9199947/?ref_=tt_urv', metadata={'source': '/review/rw9199947/?ref_=tt_urv', 'row': 37}),
  Document(page_content=": 9\nReview_Date: 19 July 2023\nAuthor: hamsterination\nRating: 6\nReview

In [ ]:
qa_with_sources_chain({"query" : "Did these movies explores themes of existentialism?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Did these movies explores themes of existentialism?',
 'result': ' Based on the reviews, it appears that "Oppenheimer" explores themes of existentialism. The review by sosrivi states that the film is "a reminder to the audience of how obsessed humans are with power and fame," which is a common theme in existentialist literature and film. Additionally, the review by chris_rowe-881-168820 states that the film is "pretty much perfect," which suggests that it effectively explores its themes and ideas.',
 'source_documents': [Document(page_content=": 117\nReview_Date: 20 July 2023\nAuthor: mattlx\nRating: 8\nReview_Title: A decent experience, still not perfect\nReview: The film's abundance of characters, combined with its non-linear narrative approach, might leave you a bit puzzled during your first viewing.\nReview_Url: /review/rw9201962/?ref_=tt_urv", metadata={'source': '/review/rw9201962/?ref_=tt_urv', 'row': 117}),
  Document(page_content=": 135\nReview_Date: 20 July 2023\nA

And with that, we have our Barbie & Oppenheimer Review RAG tool built!

This Notebook is a companion to the event put on by [AIMS](https://www.linkedin.com/company/ai-maker-space/), and [Deci](https://deci.ai/), and is authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

prompt = "Translate the following English sentence to French:"

input_ids = tokenizer.encode(prompt, return_tensors="pt")

output = model.generate(input_ids, max_length=100)

translated_text = tokenizer.decode(output[0])

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoConfig

# Specify the model name or ID of a publicly accessible model
model_id = "gpt2"

# Load model configuration
model_config = AutoConfig.from_pretrained(model_id)

# Load the model
model = GPT2LMHeadModel.from_pretrained(model_id)

# Set the model to evaluation mode
model.eval()


tokenizer = GPT2Tokenizer.from_pretrained(model_id)

generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.3,
    max_new_tokens=256
)